In [ ]:
from PIL import Image
from datetime import datetime
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization, Activation, GlobalAveragePooling2D, Lambda, Input
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import max_norm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sklearn
import tensorflow as tf
import warnings

In [ ]:
BATCH_SIZE = 64
NUM_CLASS = 200
train_dir = "/kaggle/input/ucsd-200/Train/Train"
test_dir = "/kaggle/input/ucsd-200/Test/Test"
train_txt_path = '/kaggle/input/ucsd-200/train.txt'
test_txt_path = '/kaggle/input/ucsd-200/test.txt'

#Reading the txt file
train_data, test_data = [], []
with open(train_txt_path, 'r') as train_file:
  train_data = train_file.read().splitlines()
with open(test_txt_path, 'r') as test_file:
  test_data = test_file.read().splitlines()

train_df = pd.DataFrame([x.split() for x in train_data], columns = ['filename', 'class'])
test_df = pd.DataFrame([x.split() for x in test_data], columns = ['filename', 'class'])

train_df = train_df.sort_values('filename').reset_index(drop=True)
test_df = test_df.sort_values('filename').reset_index(drop=True)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,  
    zoom_range=0.2,  
    horizontal_flip=True,  
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2,  
)


test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_augmented = train_datagen.flow_from_dataframe(
    train_df.sort_values('filename').reset_index(drop=True), 
    directory=train_dir,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True  
)


test_augmented = test_datagen.flow_from_dataframe(
    test_df.sort_values('filename').reset_index(drop=True),  
    directory=test_dir,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  
)

In [ ]:
species_names = {v: k for k, v in train_augmented.class_indices.items()}

images, labels = next(train_augmented)

plt.figure(figsize=(20, 20))

for i in range(25): 
    ax = plt.subplot(5, 5, i + 1)
    
    img = images[i] * 255.0
    plt.imshow(img.astype(np.uint8))  

    label_index = np.argmax(labels[i])
    
    plt.title(species_names[label_index])  
    plt.axis('off')  

plt.show()

In [ ]:
IMG_SHAPE = (224, 224, 3)

resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)

for layer in resnet_base.layers[-2:]:
    layer.trainable = True

model = Sequential([
    Input(shape=IMG_SHAPE),
    resnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.25),
    layers.Dense(200, activation='softmax', kernel_regularizer=l2(0.02), kernel_constraint=max_norm(3))
])

model.compile(optimizer=Adam(learning_rate=1e-06),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6, verbose=1)

model.summary()

In [ ]:
train_model = model.fit(
    train_augmented,
    validation_data=test_augmented,
    callbacks = [reduce_lr],
    epochs = 20
)

In [ ]:
# Plotting training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_model.history['accuracy'], label='Training Accuracy')
plt.plot(train_model.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1]) 
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
test_augmented.reset()

steps = int(np.ceil(test_augmented.samples / test_augmented.batch_size))

y_pred = model.predict(test_augmented, steps=steps)

y_pred_labels = np.argmax(y_pred, axis=1)

y_true = test_augmented.classes

top1_acc = accuracy_score(y_true, y_pred_labels)

conf_matrix = confusion_matrix(y_true, y_pred_labels)

class_accuracies = np.diag(conf_matrix) / np.maximum(1, conf_matrix.sum(axis=1))
avg_accuracy_per_class = np.mean(class_accuracies)

print(f"Top-1 Accuracy: {top1_acc * 100:.2f}%")
print(f"Average Accuracy per Class: {avg_accuracy_per_class * 100:.2f}%")

class_names = [species_names[i] for i in range(len(species_names))]

report = classification_report(y_true, y_pred_labels, target_names=class_names)
print("\nClassification Report:")
print(report)

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import LearningRateScheduler
import math

# Load the model
# model = load_model('/kaggle/input/1022-1810-morning-seed/tensorflow2/default/1/acc_6805 _val_accuracy_5615_0140am_1810.h5')

model.layers[-2].rate = 0.35  
model.layers[-1].kernel_regularizer = l1_l2(l1=0.01, l2=0.03)  # ElasticNet regularization

initial_lr = 2e-06

model.compile(optimizer=Adam(learning_rate=initial_lr, clipvalue=1.0),
              loss=CategoricalCrossentropy(label_smoothing=0.1),  
              metrics=['accuracy'])

total_epochs = 30 
min_lr = 5e-07  

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=min_lr, verbose=1)

train_model = model.fit(
    train_augmented,
    steps_per_epoch=train_augmented.samples, 
    validation_data=test_augmented,
    validation_steps=test_augmented.samples, 
    callbacks=[reduce_lr],
    epochs=total_epochs  
)

In [ ]:
# Plotting training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_model.history['accuracy'], label='Training Accuracy')
plt.plot(train_model.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1]) 
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
test_augmented.reset()

steps = int(np.ceil(test_augmented.samples / test_augmented.batch_size))

y_pred = model.predict(test_augmented, steps=steps)

y_pred_labels = np.argmax(y_pred, axis=1)

y_true = test_augmented.classes

top1_acc = accuracy_score(y_true, y_pred_labels)

conf_matrix = confusion_matrix(y_true, y_pred_labels)

class_accuracies = np.diag(conf_matrix) / np.maximum(1, conf_matrix.sum(axis=1))
avg_accuracy_per_class = np.mean(class_accuracies)

print(f"Top-1 Accuracy: {top1_acc * 100:.2f}%")
print(f"Average Accuracy per Class: {avg_accuracy_per_class * 100:.2f}%")

class_names = [species_names[i] for i in range(len(species_names))]

report = classification_report(y_true, y_pred_labels, target_names=class_names)
print("\nClassification Report:")
print(report)